# Redis tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [Redis commands](https://redis.io/commands)
* [Redis python client](https://redis-py.readthedocs.io/en/latest/)

Though the tutorial asks you to use Redis commands, they all have an equivalent in the Python client library.

### Import libraries

In [1]:
import redis

In [2]:
r = redis.Redis(host='localhost', port=6379)

In [3]:
r.set('foo', 'bar')
value = r.get('foo')
print(value)

b'bar'


You can launch a terminal aside, connect to your server with a Redis client and check that the value is still present :

```bash
vagrant@nosql:~$ redis-cli
127.0.0.1:6379> get foo
"bar"
```

## I. Quick start

### Strings and numbers

**Q** : Store the string `hello world` in key `greetings` with a `SET` command. Then use a `GET` command to retrieve it.

In [4]:
r.set('greetings', 'hello world')
q1 = r.get('greetings')
print(q1)

b'hello world'


**Q** : Create a `connections` key with value 0. Find the `INCR` command to increment its value, then display it. 

Try to increment the value for `greetings`.

In [11]:
r.set('connections', 0)
print(r.incr('connections'))

1


### Lists

**Q** : Create a `colors` list with strings `red`, `black` and `blue`, using the `RPUSH` command.

In [16]:
r.rpush('colors', 'red', 'black', 'blue')

12

**Q** : Display the second element of the list with `LINDEX`

In [21]:
element2 = r.lindex('colors', 1)
print(f"Le deuxième élément est : {element2}")

Le deuxième élément est : b'black'


**Q** : Display the full list with `LRANGE`

In [22]:
full_list = r.lrange('colors', 0, 12)
print(full_list)

[b'red', b'black', b'blue', b'red', b'black', b'blue', b'red', b'black', b'blue', b'red', b'black', b'blue']


### Using sets

**Q** : Create a `nosql` set containing values `redis`, `mongodb` and `orientdb`, using the `SADD` command.

In [26]:
r.sadd('nosql', 'redis', 'mongodb', 'orientdb')
ensemble = r.smembers('nosql')
print(ensemble)

{b'orientdb', b'mongodb', b'redis'}


**Q** : Test if `mysql` is inside the set, using `SISMEMBER`

In [28]:
print(r.sismember('nosql', 'mysql'))

0


**Q** : Add `hbase` to the set with `SADD`, then display all elements of the set with `SMEMBERS`. Try to add `redis` again and see what happens.

In [31]:
r.sadd('nosql', 'hbase')
ensemble2 = r.smembers('nosql')
print(ensemble2)

{b'orientdb', b'hbase', b'mongodb', b'redis'}


### Using sorted sets

Sorted Sets are similar to Redis Sets with the unique feature of values stored in a set. The difference is, every member of a Sorted Set is associated with a score, that is used in order to take the sorted set ordered, from the smallest to the greatest score.

**Q** : Using `ZADD`, create a sorted set `top14` with the following entries :

```
score city
10 Agen 
33 Bordeaux 
32 Brive 
29 Castres 
38 Clermont 
24 Grenoble 
26 La Rochelle 
32 Montpellier 
14 Oyonnax 
20 Pau 
40 Racing 
22 Stade Français 
36 Toulon 
36 Toulouse
```

In [35]:
r.zadd('top_14', {'Agen' : 10,
                'Bordeaux' : 33,
                'Brive':32,  
                'Castres':29,  
                'Clermont':38,  
                'Grenoble':24,  
                'La Rochelle':26,  
                'Montpellier':32, 
                'Oyonnax':14,  
                'Pau':20,  
                'Racing':40,  
                'Stade Français':22,  
                'Toulon': 36, 
                'Toulouse':36 })

r.zrange('top_14', 0, -1, withscores=True)


[(b'Agen', 10.0),
 (b'Oyonnax', 14.0),
 (b'Pau', 20.0),
 (b'Stade Fran\xc3\xa7ais', 22.0),
 (b'Grenoble', 24.0),
 (b'La Rochelle', 26.0),
 (b'Castres', 29.0),
 (b'Brive', 32.0),
 (b'Montpellier', 32.0),
 (b'Bordeaux', 33.0),
 (b'Toulon', 36.0),
 (b'Toulouse', 36.0),
 (b'Clermont', 38.0),
 (b'Racing', 40.0)]

**Q** : Fetch the score for `Toulon` with `ZSCORE`, and its ranking with `ZRANK`.

In [34]:
score_toulon = r.zscore('top_14', 'Toulon')
rank_toulon = r.zrank('top_14', 'Toulon')
print(f"Le score de Toulon est {score_toulon}")
print(f"Le rang de Toulon est {rank_toulon}")

Le score de Toulon est 36.0
Le rang de Toulon est 10


**Q** : `ZRANK` starts at 0 and scores are sorted from lowest to highest, so we should use the `ZREVRANK` for a true ranking of our cities.

In [37]:
classemnt_toulon = r.zrevrank('top_14', 'Toulon')
print(f"Le vrai rang de Toulon est : {classemnt_toulon}")

Le vrai rang de Toulon est : 3


**Q** : Find the commands to display :
* the 3 best teams
* teams with more than 35 points

In [42]:
best_teams = r.zrange('top_14', 11, 13)
print(f"Les 3 meilleurs équipes sont : {best_teams}")

more_35 = r.zrangebyscore('top_14', min=35, max = 1000, withscores=True)
print(f"Les équipes avec plus de 35 points sont : {more_35}")

Les 3 meilleurs équipes sont : [b'Toulouse', b'Clermont', b'Racing']
Les équipes avec plus de 35 points sont : [(b'Toulon', 36.0), (b'Toulouse', 36.0), (b'Clermont', 38.0), (b'Racing', 40.0)]


### Using dictionaries

**Q** : Create a dictionary `user:1` with `HMSET` with properties `id (1), name (Jean), age (22)`. Display it with `HGETALL`

In [43]:
r.hmset('user:1', { 'id' : 1,
                   'name' : 'Jean',
                   'age' : '22'})

r.hgetall('user:1')

c:\Anaconda3\envs\nosql\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  This is separate from the ipykernel package so we can avoid doing imports until


{b'id': b'1', b'name': b'Jean', b'age': b'22'}

**Q** : Add a `city (Lyon)` property and rename the user from `Jean` to `Paul`.

In [45]:
r.hset('user:1', 'city', 'Lyon')
r.hset('user:1', 'name', 'Paul')

r.hgetall('user:1')

{b'id': b'1', b'name': b'Paul', b'age': b'22', b'city': b'Lyon'}

## Modelling a query cache with Redis

You are modeling data from a REST query cache system with Redis.
A request is identified by the http method and its url (without the protocol).
The content of the request is stored as it is to be returned on demand.

**Q** : Insert a PUT request in the cache on http://my-api.fr/user/10 whose answer is {"id": 10, "name": "jean"}

**Q** : Insert a GET request in the cache on http://my-api.fr/user?size=2 whose answer is[{"id": 10, "name": jean}, {"id": 11, "name": "Claire"}]

**Q** : Create a set of cache request keys.

**Q** : Check if the GET request on http: //http://my-api.fr/user

**Q** : Delete the PUT request on http://my-api.fr/user/10 from the cache.

## Postquisites

The folloinwg command removes all data from your Redis cluster.

In [ ]:
!redis-cli flushall